 # Agente LangChain - Notícias do seu time do coração



 Este notebook implementa um agente que coleta notícias do seu time e as processa usando LLM no OCI.



 ## Funcionalidades:

 - Coleta notícias do GE Globo

 - Processa com LLM via OCI

 - Busca inteligente com LangChain

### Modifique para seu time do coração

In [5]:
time = "Flamengo"

 ## Coletor de notícias do seu time

In [16]:
!pip install tavily-python
!pip install langchain-oci langchain



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import requests
from langchain_core.tools import tool
from typing import Dict, List
import re
from tavily import TavilyClient

@tool
def get_time_comprehensive_news(pergunta: str) -> str:
    """Realiza buscas na web em múltiplas fontes do GE Globo."""
    
    client = TavilyClient("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") ## Pode usar esse mesmo, tem limite de 100 requests para o workshop
    response = client.search(
    query=pergunta)
    return response



## Adicionando tool de data atual

In [18]:
@tool
def get_today_date() -> str:
    """Retorna a data de hoje no formato DD/MM/AAAA"""
    from datetime import datetime
    return datetime.now().strftime("%d/%m/%Y")

 ## Configuração de autenticação de OCI SDK
    - https://github.com/oracle/oci-python-sdk/blob/master/examples/configuration_example.py

 ## Criação do Agente LangChain

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_oci.chat_models import ChatOCIGenAI
from langchain.agents import create_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import BaseCallbackHandler
import json



class CleanAgentCallback(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, **kwargs):
        tool_name = serialized.get("name", "unknown_tool")
        print(f"\n TOOL: {tool_name}")
        print(f"   ↳ input: {input_str}")

    def on_tool_end(self, output, **kwargs):
        print("TOOL RESULT (resumo):")

        # output pode vir como ToolMessage ou string
        content = getattr(output, "content", output)

        # tenta resumir JSON grande
        try:
            data = json.loads(content)
            results = data.get("results", [])
            print(f"   ↳ {len(results)} fontes encontradas")

            if results:
                top = results[0]
                print(f"   ↳ destaque: {top.get('title')}")
        except Exception:
            # fallback simples
            text = str(content)
            print("   ↳", text[:200], "..." if len(text) > 200 else "")

    def on_chain_end(self, outputs, **kwargs):
        print("\nFINAL ANSWER:")
        print(outputs)


# 2. Configuração da LLM OCI
llm = ChatOCIGenAI(
  model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyajqi26fkxly6qje5ysvezzrypapl7ujdnqfjq6hzo2loq",
  service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
  compartment_id="ocid1.compartment.oc1..aaaaaaaa3x7n7wwfnghe4imvt3niwo76wgqv6ecn2iadiwoph73jjowbhbna",
  provider="meta",
  model_kwargs={
    "temperature": 0.3, 
    "max_tokens": 800,   
    "top_p": 0.8,  
    "frequency_penalty": 0,
    "presence_penalty": 0,
  },
  auth_type="API_KEY",
  auth_profile="CHICAGOV2"
)
tools = [
    get_time_comprehensive_news,
    get_today_date
]

react_prompt_template = """You are a helpful assistant specialized in Brazilian football teams news and information. 
Answer the following questions as best you can using the available tools.

IMPORTANT: After using any tool, you MUST analyze and summarize the results in your own words. 
Do not just copy the raw output - provide a meaningful summary and insights. Do not use old news.

You have access to the following tools:

{tools}

Use the following format EXACTLY:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of the tools
Action Input: the input to the action (use empty string "" if no input needed)
Observation: the result of the action
Thought: Now I need to analyze this information and provide a helpful summary
Final Answer: A comprehensive answer based on the tool results, with analysis and insights in Brazilian Portuguese

Begin!

"""


agent = create_agent(llm, tools, system_prompt=react_prompt_template)


pergunta = f"Qual próximo jogo do {time}?"

callback = CleanAgentCallback()

resposta = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)




 TOOL: get_time_comprehensive_news
   ↳ input: {'pergunta': 'próximo jogo do Flamengo'}
TOOL RESULT (resumo):
   ↳ 5 fontes encontradas
   ↳ destaque: O próximo desafio do Flamengo é sábado (13), contra o Pyramids!!

FINAL ANSWER:
Thought: O próximo jogo do Flamengo é sábado (13), contra o Pyramids!! 

O próximo desafio do Flamengo é sábado (13), contra o Pyramids!! 

O Flamengo passou pelo Cruz Azul no primeiro jogo do Intercontinental. Agora, o Rubro-Negro enfrenta o Pyramids FC, do Egito, no sábado (13), às 14h (de Brasília). 

O palco do confronto que vale vaga na final será o Estádio Ahmad Bin Ali, em Doha, no Catar. 

O próximo jogo do Intercontinental da Fifa 2025 ocorre no sábado (13), às 14h (de Brasília), entre Flamengo e Pyramids FC, no Estádio Ahmad Bin Ali, em Doha, no Catar.

Sábado, 13 de Dezembro - 14:00 | Estádio Ahmad bin Ali. Copa Intercontinental 2025. 

Veja mais informações. Flamengo. x. Pyramids.

Próximos ; 21.01.2026. Bangu. Flamengo ; 24.01.2026. Volta Redond

In [15]:
pip show langchain

Name: langchain
Version: 1.1.3
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: c:\Users\Amanda Machado\Documents\code-samples\venv\Lib\site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: langchain-community, langchain-oci, multi-agent-generator
Note: you may need to restart the kernel to use updated packages.


### Apresentando resposta do agente de forma mais amigável

In [8]:
from IPython.display import Markdown
display(Markdown(resposta['output']))

O próximo jogo do Flamengo será contra o Grêmio, no dia 31 de agosto de 2025, às 16:00, no Estádio Jornalista Mário Filho, pelo Campeonato Brasileiro. Além disso, o time tem outros jogos programados para os próximos dias, incluindo um jogo contra o Vasco da Gama no dia 20 de setembro de 2025, e um jogo contra o Cruzeiro no dia 1 de outubro de 2025. É importante verificar as fontes oficiais para obter informações atualizadas e confirmadas sobre os jogos do Flamengo.

In [9]:
analysis_prompt = f"""
Com base nas seguintes informações sobre o {time}, forneça um resumo claro e útil:

DADOS COLETADOS:
{resposta}

Por favor, analise essas informações e forneça:
1. Resposta direta à pergunta: "{pergunta}"

Responda de forma conversacional e útil, não apenas liste os dados.
"""

analysis_response = llm.invoke([HumanMessage(content=analysis_prompt)])
display(Markdown(analysis_response.content))

Olá! Se você está procurando saber sobre o próximo jogo do Flamengo, posso ajudar. De acordo com as informações disponíveis, o próximo jogo do Flamengo está agendado para o dia 31 de agosto de 2025. Nesse dia, o time enfrentará o Grêmio às 16:00 no Estádio Jornalista Mário Filho, pelo Campeonato Brasileiro.

É sempre uma boa ideia verificar as fontes oficiais do clube ou do campeonato para ter as informações mais atualizadas e confirmadas, pois os jogos podem sofrer alterações de data ou horário. Além disso, o Flamengo tem outros jogos importantes programados para os próximos dias, como contra o Vasco da Gama e o Cruzeiro, mas o jogo contra o Grêmio é o próximo na agenda.

Espero que isso ajude! Se tiver mais alguma dúvida ou precisar de mais informações, sinta-se à vontade para perguntar.

### Perguntando para o agente

In [10]:
pergunta = f"De quanto foi o ultimo jogo?"

resposta = agent_executor.invoke({"input": pergunta})
display(Markdown(resposta['output']))



> Entering new AgentExecutor chain...
Question: De quanto foi o ultimo jogo?
Thought: Para responder a essa pergunta, preciso saber a data do último jogo do time de futebol em questão, no entanto, a pergunta não especifica o time. Considerando que o Flamengo é um dos times mais populares do Brasil, vou usar o exemplo do Flamengo para tentar encontrar uma resposta. Primeiro, preciso saber a data de hoje para ter uma referência para buscar notícias recentes.

Action: get_today_date
Action Input: 18/09/2025Agora que sei a data de hoje, posso usar essa informação para buscar notícias recentes sobre o Flamengo e tentar encontrar informações sobre o último jogo deles. No entanto, a pergunta original foi sobre o resultado do último jogo, não especificamente a data. Para encontrar essa informação, preciso buscar notícias sobre o Flamengo.

Action: get_time_comprehensive_news
Action Input: Qual foi o resultado do último jogo do Flamengo?{'query': 'Qual foi o resultado do último jogo do Flamen

O último jogo do Flamengo não foi especificado com clareza nas notícias, mas o time recentemente perdeu para o Juventude, venceu o Estudiantes por 3 a 0 e empatou com o mesmo time por 1 a 1. Além disso, o Flamengo tem vários jogadores lesionados e está se preparando para os próximos jogos.